In [1]:
import numpy as np
import pandas as pd

In [ ]:
#first, correct cancer files read count biases

In [2]:
dilution_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name='test')
dilution_calc

,healthy_file_name,healthy_read_counts,cancer_file_label,cancer_file_name,original,cancer_read_counts,correction_factor,corrected read counts
0,WHC567,1700904906,CRC-9,1279_221015,0.455,2295293721,0.741040,1.700905e+09
1,WHC567,1700904906,CRC-10,1279_241116,0.858,2331916340,0.729402,1.700905e+09
2,WHC567,1700904906,CRC-11,512_051015,0.490,2297120422,0.740451,1.700905e+09
3,WHC567,1700904906,CRC-12,512_130114,0.351,2273365787,0.748188,1.700905e+09
4,WHC568,1638484969,CRC-9,1279_221015,0.455,2295293721,0.713845,1.638485e+09
...,...,...,...,...,...,...,...,...
67,WHC565,1576440271,CRC-12,512_130114,0.351,2273365787,0.693439,1.576440e+09
68,WHC566,1698345718,CRC-9,1279_221015,0.455,2295293721,0.739925,1.698346e+09
69,WHC566,1698345718,CRC-10,1279_241116,0.858,2331916340,0.728305,1.698346e+09
70,WHC566,1698345718,CRC-11,512_051015,0.490,2297120422,0.739337,1.698346e+09


In [3]:
num_list_h = ['567', '568', '569', '570', '571', '572']
num_list_c = ['1279_221015', '1279_241116', '512_051015', '512_130114' ]

In [4]:
with open('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\\score\\healthy_list_altered.txt') as file:
    regions_h = [line.rstrip() for line in file]
len(regions_h)

144

In [5]:
regions_c = [region[3:] for region in regions_h]
len(regions_c)

144

In [6]:
for num_c in num_list_c:
    for num_h in num_list_h:
        for region in regions_c:
            fn = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_test\\selected-raw-cancer-' + \
                 num_c + '-' + region
            df = pd.read_csv(fn, header=None, sep='\t')
            factor = dilution_calc[(dilution_calc.healthy_file_name == 'WHC' + num_h) & \
                              (dilution_calc.cancer_file_name == num_c)]['correction_factor'].values[0]
            df[2] = df[2] * factor
            df.to_csv('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_test\\' + num_c + \
                      '\\WHC' + num_h + '\\' + region)

In [7]:
#second, mix!

In [7]:
file_origin_column = []
fraction_column = []
regions_dic = {}
for region_c in regions_c:
    region_h = 'chr' + region_c
    region_column = []
    for num_c in num_list_c:
        for num_h in num_list_h:
            frac_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name = num_c)
            fn_c = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_test\\' + num_c + \
                    '\\WHC' + num_h + '\\' + region_c
            fn_h = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\healthy_raw_test\\selected-raw-healthy-WHC' + \
                    num_h + '-' + region_h
            raw_c = pd.read_csv(fn_c)['2']
            raw_h = pd.read_csv(fn_h, header = None, sep = '\t')[2]
            for fraction in frac_calc['target_fraction']:
                if len(regions_dic.keys()) == 0:
                    file_origin_column.append(num_c + num_h)
                    fraction_column.append(fraction)
                
                healthy_factor = frac_calc[frac_calc.target_fraction == fraction]['healthy_factor'].values[0]
                cancer_factor = frac_calc[frac_calc.target_fraction == fraction]['cancer_factor'].values[0]
                raw_mixed = cancer_factor * raw_c + healthy_factor * raw_h
                left_flank_sum = raw_mixed[0:5000].sum()
                right_flank_sum = raw_mixed[7000:12000].sum()
                flank_mean = (left_flank_sum + right_flank_sum) / 10000
                avg = (raw_mixed[5900:6100] / flank_mean).mean()
                region_column.append(avg)
    regions_dic[region_c] = region_column

In [9]:
regions_df = pd.DataFrame(regions_dic)
regions_df['fraction'] = fraction_column
regions_df['file_origin'] = file_origin_column
regions_df

,2_90377309-90389308,10_115913313-115925312,5_132088994-132100993,12_76707915-76719914,9_111201851-111213850,11_94314325-94326324,1_86102052-86114051,4_49653612-49665611,11_4191008-4203007,10_34710330-34722329,...,X_10898615-10910614,5_7165158-7177157,4_4085940-4097939,4_120310198-120322197,7_152659206-152671205,6_55727191-55739190,6_47483844-47495843,2_163679216-163691215,fraction,file_origin
0,3.950934,0.859894,0.917641,1.038268,1.294593,1.180603,1.150593,20.376236,0.829391,0.886040,...,0.914825,1.275918,1.006567,1.167895,0.987504,1.265758,1.303905,1.053538,0.000,1279_221015567
1,3.947053,0.857450,0.917101,1.036359,1.289058,1.177961,1.149185,20.351061,0.828723,0.887343,...,0.911221,1.273099,1.003890,1.165809,0.988098,1.263035,1.297960,1.053135,0.005,1279_221015567
2,3.943154,0.854996,0.916557,1.034441,1.283495,1.175306,1.147772,20.325797,0.828053,0.888646,...,0.907597,1.270289,1.001204,1.163710,0.988693,1.260314,1.292018,1.052730,0.010,1279_221015567
3,3.939236,0.852534,0.916010,1.032515,1.277902,1.172638,1.146354,20.300443,0.827381,0.889949,...,0.903954,1.267488,0.998511,1.161600,0.989290,1.257594,1.286080,1.052324,0.015,1279_221015567
4,3.935300,0.850062,0.915460,1.030581,1.272280,1.169958,1.144930,20.274999,0.826708,0.891252,...,0.900292,1.264695,0.995808,1.159477,0.989889,1.254876,1.280144,1.051917,0.020,1279_221015567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,3.531476,0.739854,1.026146,1.066416,0.915780,1.033521,0.990457,20.385478,1.037404,1.041165,...,0.691568,1.319994,0.837628,0.951747,1.096812,1.007936,0.824440,1.048959,0.300,512_130114572
698,3.502670,0.719334,1.022577,1.025413,0.873199,1.003940,0.992547,20.269242,1.021528,1.008965,...,0.662217,1.280430,0.831854,0.912882,1.101972,1.005277,0.790184,1.035464,0.325,512_130114572
699,3.470146,0.698326,1.018874,0.983029,0.830375,0.973948,0.994738,20.149909,1.005218,0.976051,...,0.633179,1.239393,0.825786,0.871571,1.107145,1.002633,0.755489,1.022062,0.350,512_130114572
700,3.433136,0.676809,1.015027,0.939192,0.787306,0.943536,0.997036,20.027355,0.988458,0.942399,...,0.604448,1.196800,0.819400,0.827577,1.112330,1.000002,0.720344,1.008754,0.375,512_130114572


In [10]:
regions_df.to_csv('test_new.csv')